In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
from pandas import to_datetime
import seaborn as sn
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr, iqr

In [2]:
customer_indicators = pd.read_csv("../../indicators.csv", index_col=0)
customer_indicators

,I,Iu,spending,Imax,avg_bought,baskets,E
CustomerID,,,,,,,
12347.0,0.108021,0.059890,0.124483,0.217940,0.019291,0.031579,0.628584
12348.0,0.002658,0.003964,0.071554,0.002658,0.274122,0.000000,0.000000
12349.0,0.307641,0.311199,0.368923,0.307641,0.023277,0.000000,0.585006
12350.0,0.112957,0.073340,0.081727,0.112957,0.008866,0.000000,0.393407
12352.0,0.044423,0.037944,0.064072,0.067110,0.033188,0.031579,0.551425
...,...,...,...,...,...,...,...
18280.0,0.028571,0.048563,0.053783,0.028571,0.030539,0.000000,0.324265
18281.0,0.033223,0.023786,0.013787,0.033223,0.005147,0.000000,0.230642
18282.0,0.016279,0.021308,0.020487,0.017276,0.025711,0.005263,0.299490


In [3]:
def target_label(row):
    first_third = customer_indicators["spending"].quantile(0.33)
    second_third = customer_indicators["spending"].quantile(0.66)
    if (row["spending"] <= first_third):
        return 1
    elif (row["spending"] > first_third and row["spending"] <= second_third):
        return 2
    return 3

customer_indicators["label"] = customer_indicators.apply(target_label, axis=1)

In [29]:
customer_indicators

,I,Iu,spending,Imax,avg_bought,baskets,E,label
CustomerID,,,,,,,,
12347.0,0.108021,0.059890,0.124483,0.217940,0.019291,0.031579,0.628584,3
12348.0,0.002658,0.003964,0.071554,0.002658,0.274122,0.000000,0.000000,2
12349.0,0.307641,0.311199,0.368923,0.307641,0.023277,0.000000,0.585006,3
12350.0,0.112957,0.073340,0.081727,0.112957,0.008866,0.000000,0.393407,3
12352.0,0.044423,0.037944,0.064072,0.067110,0.033188,0.031579,0.551425,2
...,...,...,...,...,...,...,...,...
18280.0,0.028571,0.048563,0.053783,0.028571,0.030539,0.000000,0.324265,2
18281.0,0.033223,0.023786,0.013787,0.033223,0.005147,0.000000,0.230642,1
18282.0,0.016279,0.021308,0.020487,0.017276,0.025711,0.005263,0.299490,1


## Classificazione con Random Forest

In [46]:
from sklearn.model_selection import train_test_split

customer_indicatorsForClass=customer_indicators.copy()

#extract and remove original label values
originalLabel=customer_indicatorsForClass["label"].values
customer_indicatorsForClass=customer_indicatorsForClass.drop(columns=['label'])
#generate training and test set
train_set, test_set, train_label, test_label = train_test_split(customer_indicatorsForClass, originalLabel, stratify =originalLabel, test_size=0.40)

In [47]:
print(train_set)
print(train_label.shape)
print(test_set)
print(test_label.shape)

                   I        Iu  spending      Imax  avg_bought   baskets  \
CustomerID                                                                 
13932.0     0.045183  0.068385  0.083432  0.045183    0.032139  0.000000   
14422.0     0.161130  0.124546  0.165934  0.288372    0.020156  0.026316   
15464.0     0.145515  0.316155  0.152503  0.145515    0.015725  0.000000   
14849.0     0.053267  0.024818  0.058210  0.153488    0.020222  0.121053   
15234.0     0.065116  0.053518  0.053304  0.065116    0.012052  0.000000   
...              ...       ...       ...       ...         ...       ...   
15527.0     0.039784  0.040510  0.065010  0.064452    0.023994  0.036842   
12867.0     0.235216  0.146255  0.126745  0.334884    0.007381  0.031579   
14201.0     0.019364  0.018123  0.021121  0.033223    0.018237  0.031579   
14412.0     0.059801  0.053518  0.067213  0.062458    0.024839  0.005263   
12394.0     0.056811  0.048563  0.076946  0.108970    0.023812  0.005263   

           

## Grid search per la ricerca dei parametri migliori

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

#define the parameters' values you want to try
param_dist = {"max_depth": [2,3,4,5,6,7,10,12,None],
              "max_features": [2,3],
              "min_samples_split": sp_randint(10, 100),
              "min_samples_leaf": sp_randint(10, 100),
              "bootstrap": [True, False],
              "criterion": ["entropy", "gini"],
              "class_weight":['balanced', None]}
#define the number of iters
n_iter_search = 50
#define the model
clf = RandomForestClassifier(n_estimators=30)
#define the grid search
grid_search = RandomizedSearchCV(clf, param_distributions=param_dist, 
                            n_iter=n_iter_search, 
                            n_jobs=10, 
                            scoring=make_scorer(accuracy_score))
#run the grid search
grid_search.fit(train_set, train_label)

print('Best setting parameters ', grid_search.cv_results_['params'][0])
print('Mean and std of this setting ', grid_search.cv_results_['mean_test_score'][0], 
      grid_search.cv_results_['std_test_score'][0])


Best setting parameters  {'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 90, 'min_samples_split': 13}
Mean and std of this setting  0.9911517296550386 0.006803159493211003


In [49]:
rf = RandomForestClassifier(n_estimators=30, 
                             criterion='entropy',
                             max_features=3,
                             max_depth=3, 
                             min_samples_split=13,
                             min_samples_leaf=90,
                             bootstrap=False) 
rf = rf.fit(train_set, train_label)

In [50]:
#prediction on the test test
test_pred_rf = rf.predict(test_set)

In [51]:
from sklearn.metrics import classification_report
def report_scores(test_label, test_pred):
    print(classification_report(test_label, 
                            test_pred, 
                            target_names=["1","2","3"]))
#compute the performance of the model
report_scores(test_label, test_pred_rf)

              precision    recall  f1-score   support

           1       1.00      0.97      0.99       547
           2       0.97      1.00      0.98       547
           3       1.00      1.00      1.00       564

    accuracy                           0.99      1658
   macro avg       0.99      0.99      0.99      1658
weighted avg       0.99      0.99      0.99      1658

